In [1]:
import pandas as pd
import numpy as np

# no scientific notation
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# Get and Clean Occupation codes from website

In [2]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context # to fix a certificate error
occ = pd.read_html('https://usa.ipums.org/usa/volii/occ2018.shtml')[1]
occ.columns = ['code', 'occ'] # occ.columns.droplevel(1)
occ

,code,occ
0,000,N/A (not applicable)
1,"Management, Business, Science, and Arts Occupa...","Management, Business, Science, and Arts Occupa..."
2,"Management, Business, and Financial Occupations:","Management, Business, and Financial Occupations:"
3,0010,Chief executives and legislators
4,0020,General and operations managers
...,...,...
542,9800,Military officer special and tactical operatio...
543,9810,First-line enlisted military supervisors
544,9825,Military enlisted tactical operations and air/...
545,9830,"Military, rank not specified"


In [3]:
# The values in column OCC that are not digits are headers for occupation groups
# mark and fill forward to create a new column that contains the occupation group for each occupation
occ['group'] = occ['code'].apply(lambda x: x if not x.isdigit() else np.nan)
occ['group'] = occ['group'].ffill()
occ

,code,occ,group
0,000,N/A (not applicable),NaN
1,"Management, Business, Science, and Arts Occupa...","Management, Business, Science, and Arts Occupa...","Management, Business, Science, and Arts Occupa..."
2,"Management, Business, and Financial Occupations:","Management, Business, and Financial Occupations:","Management, Business, and Financial Occupations:"
3,0010,Chief executives and legislators,"Management, Business, and Financial Occupations:"
4,0020,General and operations managers,"Management, Business, and Financial Occupations:"
...,...,...,...
542,9800,Military officer special and tactical operatio...,Transportation and Material Moving Occupations:
543,9810,First-line enlisted military supervisors,Transportation and Material Moving Occupations:
544,9825,Military enlisted tactical operations and air/...,Transportation and Material Moving Occupations:
545,9830,"Military, rank not specified",Transportation and Material Moving Occupations:


In [4]:
# final cleaning for occupation data

occ = occ[occ['code'].str.isdigit()].copy() # remove the headers
occ.loc[:,'code'] = occ['code'].astype(int) # convert the occupation code to integer
occ.loc[:,'group'] = occ['group'].str.strip() # remove the trailing spaces
occ.loc[:,'group'] = occ['group'].str.replace(':', '') # remove the colon
occ = occ.fillna('Not Applicable') # fill the NA occupation with 'Not Applicable'
occ

,code,occ,group
0,0,N/A (not applicable),Not Applicable
3,10,Chief executives and legislators,"Management, Business, and Financial Occupations"
4,20,General and operations managers,"Management, Business, and Financial Occupations"
5,40,Advertising and promotions managers,"Management, Business, and Financial Occupations"
6,51,Marketing managers,"Management, Business, and Financial Occupations"
...,...,...,...
542,9800,Military officer special and tactical operatio...,Transportation and Material Moving Occupations
543,9810,First-line enlisted military supervisors,Transportation and Material Moving Occupations
544,9825,Military enlisted tactical operations and air/...,Transportation and Material Moving Occupations
545,9830,"Military, rank not specified",Transportation and Material Moving Occupations


# Import Main Dataset

In [5]:
with open('ACSIncome_state_number.arff', 'r') as f:
    lines = f.readlines()[1:13] # get the feature names from the first 13 lines

columns = [words.split()[1] for words in lines] # extract the feature names

df = pd.read_csv('ACSIncome_state_number.arff', skiprows=14, names=columns)
df

,AGEP,COW,SCHL,MAR,OCCP,POBP,RELP,WKHP,SEX,RAC1P,ST,PINCP
0,18.000,1.000,18.000,5.000,4720.000,13.000,17.000,21.000,2.000,2.000,1.000,1600.000
1,53.000,5.000,17.000,5.000,3605.000,18.000,16.000,40.000,1.000,1.000,1.000,10000.000
2,41.000,1.000,16.000,5.000,7330.000,1.000,17.000,40.000,1.000,1.000,1.000,24000.000
3,18.000,6.000,18.000,5.000,2722.000,1.000,17.000,2.000,2.000,1.000,1.000,180.000
4,21.000,5.000,19.000,5.000,3870.000,12.000,17.000,50.000,1.000,1.000,1.000,29000.000
...,...,...,...,...,...,...,...,...,...,...,...,...
1664495,39.000,6.000,16.000,5.000,6260.000,72.000,0.000,20.000,1.000,1.000,72.000,9600.000
1664496,38.000,6.000,14.000,5.000,4251.000,72.000,0.000,32.000,1.000,8.000,72.000,2400.000
1664497,37.000,1.000,19.000,3.000,7750.000,17.000,13.000,40.000,2.000,9.000,72.000,19700.000
1664498,47.000,1.000,16.000,1.000,8990.000,72.000,1.000,40.000,1.000,8.000,72.000,18700.000


In [6]:
workclass = {
    'b': 'N/A (less than 16 years old/NILF who last worked more than 5 years ago or never worked)',
    '1': 'Employee of a private for-profit company or business, or of an individual, for wages, salary, or commissions',
    '2': 'Employee of a private not-for-profit, tax-exempt, or charitable organization',
    '3': 'Local government employee (city, county, etc.)',
    '4': 'State government employee',
    '5': 'Federal government employee',
    '6': 'Self-employed in own not incorporated business, professional practice, or farm',
    '7': 'Self-employed in own incorporated business, professional practice or farm',
    '8': 'Working without pay in family business or farm',
    '9': 'Unemployed and last worked 5 years ago or earlier or never worked'
}
df.loc[:, 'COW'] = df.COW.astype(str).str.replace('.0', '').map(workclass).astype('category')

C:\Users\brady\AppData\Local\Temp\ipykernel_23148\354053017.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:, 'COW'] = df.COW.astype(str).str.replace('.0', '').map(workclass).astype('category')


In [7]:
school = {
    1: 'No schooling completed',
    2: 'Nursery school, preschool',
    3: 'Kindergarten',
    4: 'Grade 1',
    5: 'Grade 2',
    6: 'Grade 3',
    7: 'Grade 4',
    8: 'Grade 5',
    9: 'Grade 6',
    10: 'Grade 7',
    11: 'Grade 8',
    12: 'Grade 9',
    13: 'Grade 10',
    14: 'Grade 11',
    15: '12th grade - no diploma',
    16: 'Regular high school diploma',
    17: 'GED or alternative credential',
    18: 'Some college, but less than 1 year',
    19: '1 or more years of college credit, no degree',
    20: "Associate's degree",
    21: "Bachelor's degree",
    22: "Master's degree",
    23: "Professional degree beyond a bachelor's degree",
    24: "Doctorate degree"
}
df.loc[:, 'SCHL'] = df.SCHL.astype(int).map(school).astype('category')

In [8]:
marriage = {
    1:'Married',
    2: 'Widowed',
    3: 'Divorced',
    4: 'Separated',
    5: 'Never married or under 15 years old'
}
df.loc[:, 'MAR'] = df.MAR.astype(int).map(marriage).astype('category')

In [9]:
# get a map of code to occupation category
occupation = dict(zip(occ.code, occ.group))

df.loc[:, 'OCCP'] = df.OCCP.astype(int).map(occupation).astype('category')

In [10]:
birth_place = {
1: 'Alabama/AL',
2: 'Alaska/AK',
4: 'Arizona/AZ',
5: 'Arkansas/AR',
6: 'California/CA',
8: 'Colorado/CO',
9: 'Connecticut/CT',
10: 'Delaware/DE',
11: 'District of Columbia/DC',
12: 'Florida/FL',
13: 'Georgia/GA',
15: 'Hawaii/HI',
16: 'Idaho/ID',
17: 'Illinois/IL',
18: 'Indiana/IN',
19: 'Iowa/IA',
20: 'Kansas/KS',
21: 'Kentucky/KY',
22: 'Louisiana/LA',
23: 'Maine/ME',
24: 'Maryland/MD',
25: 'Massachusetts/MA',
26: 'Michigan/MI',
27: 'Minnesota/MN',
28: 'Mississippi/MS',
29: 'Missouri/MO',
30: 'Montana/MT',
31: 'Nebraska/NE',
32: 'Nevada/NV',
33: 'New Hampshire/NH',
34: 'New Jersey/NJ',
35: 'New Mexico/NM',
36: 'New York/NY',
37: 'North Carolina/NC',
38: 'North Dakota/ND',
39: 'Ohio/OH',
40: 'Oklahoma/OK',
41: 'Oregon/OR',
42: 'Pennsylvania/PA',
44: 'Rhode Island/RI',
45: 'South Carolina/SC',
46: 'South Dakota/SD',
47: 'Tennessee/TN',
48: 'Texas/TX',
49: 'Utah/UT',
50: 'Vermont/VT',
51: 'Virginia/VA',
53: 'Washington/WA',
54: 'West Virginia/WV',
55: 'Wisconsin/WI',
56: 'Wyoming/WY',
60: 'American Samoa',
66: 'Guam',
69: 'Commonwealth of the Northern Mariana Islands',
72: 'Puerto Rico',
78: 'US Virgin Islands',
100: 'Albania',
102: 'Austria',
103: 'Belgium',
104: 'Bulgaria',
105: 'Czechoslovakia',
106: 'Denmark',
108: 'Finland',
109: 'France',
110: 'Germany',
116: 'Greece',
117: 'Hungary',
118: 'Iceland',
119: 'Ireland',
120: 'Italy',
126: 'Netherlands',
127: 'Norway',
128: 'Poland',
129: 'Portugal',
130: 'Azores Islands',
132: 'Romania',
134: 'Spain',
136: 'Sweden',
137: 'Switzerland',
138: 'United Kingdom, Not Specified',
139: 'England',
140: 'Scotland',
142: 'Northern Ireland',
147: 'Yugoslavia',
148: 'Czech Republic',
149: 'Slovakia',
150: 'Bosnia and Herzegovina',
151: 'Croatia',
152: 'Macedonia',
154: 'Serbia',
156: 'Latvia',
157: 'Lithuania',
158: 'Armenia',
159: 'Azerbaijan',
160: 'Belarus',
161: 'Georgia',
162: 'Moldova',
163: 'Russia',
164: 'Ukraine',
165: 'USSR',
166: 'Europe',
167: 'Kosovo',
168: 'Montenegro',
169: 'Other Europe, Not Specified',
200: 'Afghanistan',
202: 'Bangladesh',
203: 'Bhutan',
205: 'Myanmar',
206: 'Cambodia',
207: 'China',
209: 'Hong Kong',
210: 'India',
211: 'Indonesia',
212: 'Iran',
213: 'Iraq',
214: 'Israel',
215: 'Japan',
216: 'Jordan',
217: 'Korea',
218: 'Kazakhstan',
219: 'Kyrgyzstan',
222: 'Kuwait',
223: 'Laos',
224: 'Lebanon',
226: 'Malaysia',
228: 'Mongolia',
229: 'Nepal',
231: 'Pakistan',
233: 'Philippines',
235: 'Saudi Arabia',
236: 'Singapore',
238: 'Sri Lanka',
239: 'Syria',
240: 'Taiwan',
242: 'Thailand',
243: 'Turkey',
245: 'United Arab Emirates',
246: 'Uzbekistan',
247: 'Vietnam',
248: 'Yemen',
249: 'Asia',
253: 'South Central Asia, Not Specified',
254: 'Other Asia, Not Specified',
300: 'Bermuda',
301: 'Canada',
303: 'Mexico',
310: 'Belize',
311: 'Costa Rica',
312: 'El Salvador',
313: 'Guatemala',
314: 'Honduras',
315: 'Nicaragua',
316: 'Panama',
321: 'Antigua & Barbuda',
323: 'Bahamas',
324: 'Barbados',
327: 'Cuba',
328: 'Dominica',
329: 'Dominican Republic',
330: 'Grenada',
332: 'Haiti',
333: 'Jamaica',
338: 'St. Kitts-Nevis',
339: 'St. Lucia',
340: 'St. Vincent & the Grenadines',
341: 'Trinidad & Tobago',
343: 'West Indies',
344: 'Caribbean, Not Specified',
360: 'Argentina',
361: 'Bolivia',
362: 'Brazil',
363: 'Chile',
364: 'Colombia',
365: 'Ecuador',
368: 'Guyana',
369: 'Paraguay',
370: 'Peru',
372: 'Uruguay',
373: 'Venezuela',
374: 'South America',
399: 'Americas, Not Specified',
400: 'Algeria',
407: 'Cameroon',
408: 'Cabo Verde',
412: 'Congo',
414: 'Egypt',
416: 'Ethiopia',
417: 'Eritrea',
420: 'Gambia',
421: 'Ghana',
423: 'Guinea',
425: 'Ivory Coast',
427: 'Kenya',
429: 'Liberia',
430: 'Libya',
436: 'Morocco',
440: 'Nigeria',
442: 'Rwanda',
444: 'Senegal',
447: 'Sierra Leone',
448: 'Somalia',
449: 'South Africa',
451: 'Sudan',
453: 'Tanzania',
454: 'Togo',
456: 'Tunisia',
457: 'Uganda',
459: 'Democratic Republic of Congo (Zaire)',
460: 'Zambia',
461: 'Zimbabwe',
462: 'Africa',
463: 'South Sudan',
464: 'Northern Africa, Not Specified',
467: 'Western Africa, Not Specified',
468: 'Other Africa, Not Specified',
469: 'Eastern Africa, Not Specified',
501: 'Australia',
508: 'Fiji',
511: 'Marshall Islands',
512: 'Micronesia',
515: 'New Zealand',
523: 'Tonga',
527: 'Samoa',
554: 'Other US Island Areas, Oceania, Not Specified, or at Sea'
}

In [11]:
df.loc[:, 'POBP'] = df.POBP.astype(int).map(birth_place).astype('category')

In [12]:
# RELP
relationship = {
    0: 'Reference person',
    1: 'Husband/wife',
    2: 'Biological son or daughter',
    3: 'Adopted son or daughter',
    4: 'Stepson or stepdaughter',
    5: 'Brother or sister',
    6: 'Father or mother',
    7: 'Grandchild',
    8: 'Parent-in-law',
    9: 'Son-in-law or daughter-in-law',
    10: 'Other relative',
    11: 'Roomer or boarder',
    12: 'Housemate or roommate',
    13: 'Unmarried partner',
    14: 'Foster child',
    15: 'Other nonrelative',
    16: 'Institutionalized group quarters population',
    17: 'Noninstitutionalized group quarters population'
}

df.loc[:, 'RELP'] = df.RELP.astype(int).map(relationship).astype('category')

In [13]:
# SEX

df.loc[:, 'SEX'] = df.SEX.astype(int).map({1:'Male', 2:'Female'}).astype('category')

In [14]:
# RAC1P

race = {
    1: 'White alone',
    2: 'Black or African American alone',
    3: 'American Indian alone',
    4: 'Alaska Native alone',
    5: 'American Indian and Alaska Native tribes specified; or American Indian or Alaska Native, not specified and no other races',
    6: 'Asian alone',
    7: 'Native Hawaiian and Other Pacific Islander alone',
    8: 'Some Other Race alone',
    9: 'Two or More Races'
}

df.loc[:, 'RAC1P'] = df.RAC1P.astype(int).map(race).astype('category')

In [15]:
# ST
# The same map can be used as for POBP because state codes are the same in both datasets

df.loc[:, 'ST'] = df.ST.astype(int).map(birth_place).astype('category')

In [17]:
# convert all floats to ints
floats = df.select_dtypes(include=['float64']).columns
df[floats] = df[floats].astype(int)

In [18]:
df.dtypes

AGEP        int32
COW      category
SCHL     category
MAR      category
OCCP     category
POBP     category
RELP     category
WKHP        int32
SEX      category
RAC1P    category
ST       category
PINCP       int32
dtype: object

In [19]:
df.isna().sum()

AGEP     0
COW      0
SCHL     0
MAR      0
OCCP     0
POBP     0
RELP     0
WKHP     0
SEX      0
RAC1P    0
ST       0
PINCP    0
dtype: int64

In [20]:
df.to_parquet('cleaned_data.parquet')